In [1]:
import sys
sys.path.insert(0, '../inference')

import pytorch_lightning as pl
import bitsandbytes as bnb
import torch
from src.rudolph.model.utils import get_attention_mask

from gym.spaces import Box
import numpy as np
import cv2
import matplotlib.pyplot as plt
from deep_rl_zoo.networks.dqn import RainbowDqnMlpNet, RainbowDqnConvNet, R2d2DqnConvNet
from deep_rl_zoo import main_loop
from deep_rl_zoo.checkpoint import PyTorchCheckpoint
from deep_rl_zoo import gym_env
from deep_rl_zoo import greedy_actors
import deep_rl_zoo.types as types_lib
import random
from moviepy.editor import ImageSequenceClip
import pandas as pd

from omegaconf import OmegaConf
from src.rudolph.model import get_rudolph_model
from src.inference.dataloader import DatasetRetriever
from src.inference.inference import run_inference
from torch.utils.data import DataLoader
#VQA
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import nltk
import wordtodigits as w2d
from data_for_inference.vqa_metrics import calc_meteor
import json
# Generation
import os
import torch
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from data_for_inference.generation_metrics.fid import calculate_fid_given_paths
from data_for_inference.generation_metrics.inception import InceptionV3
from data_for_inference.generation_metrics.ruclip import calc_ruclip_metric
import json
# TextQA
import argparse
from data_for_inference.textqa_metrics.calculate_f1 import get_f1
import json
def read_json(file_name):
    with open(file_name) as f:
        result = json.load(f)
    return result

from torch.utils.data import DataLoader
from src.train.dataloader_custom import DatasetRetriever, fb_collate_fn, DEFAULT_SPC_TOKENS  ########### dataloader_custom
from src.train.utils import create_dataset
from rudalle import get_tokenizer, get_vae
from catalyst.data import BalanceClassSampler, DistributedSamplerWrapper
import pandas as pd
import webdataset as wds
import s3dataset
from PIL import Image
import io
import torchvision.transforms as T
from pytorch_lightning.trainer.supporters import CombinedLoader

from src.inference.inference_api import ruDolphApi
from tqdm import tqdm

2023-01-27 00:25:00.025926: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 00:25:02.136134: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-27 00:25:02.136226: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-27 00:25:02.136234: W tensorfl


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 111
CUDA SETUP: Loading binary /home/jovyan/.imgenv-rl-models-a100-8-0/lib/python3.7/site-packages/bitsandbytes/libbitsandbytes_cuda111.so...


/home/jovyan/.imgenv-rl-models-a100-8-0/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: /home/jovyan/.imgenv-rl-models-a100-8-0 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/jovyan/.imgenv-rl-models-a100-8-0/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/home/jovyan/.imgenv-rl-models-a100-8-0/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/home/jovyan/.imgenv-rl-models-a100-8-0/lib/python3.7/site-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('80'), Pos

In [2]:
storage_options = {
    "anon": False,
    'key': 'officecds-user01',
    'secret':'jym0FuboUnR5VsPmCgYTGv1QQfglYZhPRWbEfS59',
    'client_kwargs': {
        'endpoint_url':'https://s3pd12.sbercloud.ru'
    }
}
s3dataset.init_webdataset(storage_options)